### first part when substitutions json-storage created was writed to script `../sourse/mutations_extractor.py`

- 2nd part when mut-spec table created exist only here

In [1]:
import json
from queue import Queue
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ete3 import PhyloTree

path_to_tree = "../data/mulal_gisaid_2021-01-22.filtered.fasta.prank.anc.dnd"
path_to_fasta = "../data/mulal_gisaid_2021-01-22.filtered.fasta.prank.anc.fas"

In [2]:
def read_fasta_generator(filepath):
    """read fasta without deleting '\n' from line ends to write that
    in the next step
    """
    with open(filepath) as fin:
        seq = ''
        for line in fin:
            if line.startswith(">"):
                if seq != '':
                    yield header, seq
                header = line
                seq = ''
            else:
                seq += line
        yield header, seq


fasta_generator = read_fasta_generator(path_to_fasta)
fasta_storage = dict()


def get_sequence(node_name):
    """read fasta file online and write seqs to dict if another one are needed"""
    if node_name in fasta_storage:
        seq = fasta_storage[node_name]
        return seq
    
    for new_node_name, new_seq in fasta_generator:
        new_node_name = new_node_name.strip(">\n")
        new_seq = new_seq.replace("\n", "")
        fasta_storage[new_node_name] = new_seq
        if new_node_name == node_name:
            return new_seq


def node_parent(node):
    try: return next(node.iter_ancestors())
    except: return None


def trim_two_seqs(seq1, seq2):
    """there are '----' in the start and end of seqs. Drop it!"""
    n = len(seq1)
    start_pos, stop_pos = 0, n
    for start_pos in range(n):
        if not (seq1[start_pos] == '-' or seq2[start_pos] == "-"):
            break
    
    for stop_pos in range(n - 1, -1, -1):
        if not (seq1[stop_pos] == '-' or seq2[stop_pos] == "-"):
            break
    stop_pos += 1
    return start_pos, stop_pos


def release_mutations_from_two_seqs(parent_seq: str, child_seq: str):
    start_pos, stop_pos = trim_two_seqs(parent_seq, child_seq)
    
    mutations = []
    for pos in range(start_pos, stop_pos):
        sourse_nucl = parent_seq[pos]
        mutant_nucl = child_seq[pos]
        if sourse_nucl != mutant_nucl:
            mutations.append((pos, sourse_nucl, mutant_nucl))
    return mutations


def get_mutations(node):
    parent = node_parent(node)
    if parent is None:
        return
    seq_of_parent = get_sequence(parent.name)
    seq_of_child = get_sequence(node.name)
    assert len(seq_of_child) == len(seq_of_parent), (
        "parent and child seq lenghts aren't equal"
    )
    mutations = release_mutations_from_two_seqs(
        seq_of_parent,
        seq_of_child,
    )
    return parent.name, node.name, mutations

In [3]:
tree = PhyloTree(path_to_tree, format=1)

In [ ]:
# BFS
# def bfs_to_extract_mutspec():
discovered_nodes = set()
Q = Queue()
Q.put(tree)
discovered_nodes.add(tree.name)

overall_mutations = []
while not Q.empty():
    cur_node = Q.get()
    for chi in cur_node.children:
        Q.put(chi)
    
    if cur_node.name not in discovered_nodes:
        discovered_nodes.add(cur_node.name)
        # main process starts here 
        cur_mutations_of_one = get_mutations(cur_node)
        overall_mutations.append(cur_mutations_of_one)
        

In [ ]:
len(discovered_nodes), len(tree.get_descendants()), len(overall_mutations)

In [ ]:
overall_mutations[100000]

In [6]:
# with open('../data/overall_mutations.json', 'w') as fout:
#     json.dump(overall_mutations, fout)

-----------------

In [2]:
with open('../data/overall_mutations_2.json') as fin:
    overall_mutations = json.load(fin)

In [3]:
len(overall_mutations)

100976

In [4]:
([x for x in overall_mutations if len(x) > 0 and x[0][1] == '-'])

[]

In [5]:
nucl_idx = dict()
variants = "ACGT-"
i = 0
for char1 in variants:
    for char2 in variants.replace(char1, ''):
        # print(i, f"{char1}2{char2}")
        nucl_idx[f"{char1}2{char2}"] = i
        i += 1
print(nucl_idx)

{'A2C': 0, 'A2G': 1, 'A2T': 2, 'A2-': 3, 'C2A': 4, 'C2G': 5, 'C2T': 6, 'C2-': 7, 'G2A': 8, 'G2C': 9, 'G2T': 10, 'G2-': 11, 'T2A': 12, 'T2C': 13, 'T2G': 14, 'T2-': 15, '-2A': 16, '-2C': 17, '-2G': 18, '-2T': 19}


In [6]:
mutation_matrix = np.zeros((29903, len(nucl_idx)), dtype=np.uint32)

for mutations in overall_mutations:
    for pos, source_nucl, mutant_nucl in mutations[-1]:
        code = f"{source_nucl}2{mutant_nucl}"
        mutation_matrix[pos, nucl_idx[code]] += 1

In [7]:
df = pd.DataFrame(mutation_matrix, columns=nucl_idx.keys())
df.head()

,A2C,A2G,A2T,A2-,C2A,C2G,C2T,C2-,G2A,G2C,G2T,G2-,T2A,T2C,T2G,T2-,-2A,-2C,-2G,-2T
0,14,9,17,0,14,1,0,0,13,1,0,0,47,0,0,0,0,0,0,0
1,1,0,53,0,1,0,33,0,0,0,20,0,34,14,12,0,0,0,0,0
2,0,0,51,0,0,0,15,0,0,0,53,0,23,7,38,0,0,0,0,0
3,15,49,59,0,24,2,0,0,67,0,3,0,107,0,2,0,0,0,0,0
4,7,27,48,0,10,0,1,0,66,0,2,0,80,0,4,0,0,0,0,0


In [8]:
df.reset_index().to_csv("../data/gisaid_sample_mutspec2.csv", index=None)

In [7]:
refseq = get_sequence("NC_045512.2")
len(refseq)

29903

In [9]:
seq_lens = []
for _, seq in fasta_generator:
    seq_lens.append(len(seq.strip("\n-")))

In [10]:
import pandas as pd
pd.Series(seq_lens).value_counts().iloc[:20]

29782    5394
29903    3228
29836     333
29775     292
29902     281
29899     266
29882     265
29901     255
29840     236
29866     234
29865     231
29864     221
29862     208
29873     207
29813     197
29868     183
29857     177
29859     175
29872     166
29863     165
dtype: int64